¡Excelente! Usar CBRKit en un Notebook (Jupyter o Google Colab) es la mejor forma de prototipar, ya que puedes visualizar la tabla de casos y los resultados de similitud de forma inmediata.

Aquí tienes una guía paso a paso para implementar un motor de CBR funcional en una celda.

1. Instalación y Preparación

In [4]:
pip install cbrkit

Note: you may need to restart the kernel to use updated packages.


El error ocurre porque CBRKit no instala automáticamente todas las librerías de algoritmos de comparación (como Levenshtein) para mantenerse ligero.

In [5]:
pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.6/153.6 kB 585.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 1.9 MB/s eta 0:00:0000:0100:010m
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install "cbrkit[all]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 381.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 340.0 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 3.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 618.2 kB/s eta 0:00:000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 1.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.3/32.3 MB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB ? eta 0:00:000:00:0100:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2. Definición de la Base de Casos
Vamos a crear un ejemplo de Soporte Técnico de Software. Imaginemos que queremos resolver errores de usuarios basados en experiencias pasadas.

In [4]:
import cbrkit
import pandas as pd

# 1. Definimos nuestros casos previos
casos_data = {
    "caso1": {"error": "pantalla_azul", "memoria": 8, "uso_cpu": 90, "solucion": "Reiniciar drivers de video"},
    "caso2": {"error": "lags_intermitentes", "memoria": 4, "uso_cpu": 50, "solucion": "Limpiar archivos temporales"},
    "caso3": {"error": "pantalla_azul", "memoria": 16, "uso_cpu": 95, "solucion": "Revisar hardware de RAM"},
}

# 2. La consulta (el problema actual del usuario)
query = {"error": "pantalla_azul", "memoria": 12, "uso_cpu": 85}

import cbrkit

# Definimos la función de similitud con el parámetro corregido
sim_func = cbrkit.sim.generic.table(
    attributes={
        "error": cbrkit.sim.strings.levenshtein(),
        # Cambiamos 'max_distance' por 'max'
        "memoria": cbrkit.sim.numbers.linear(max=12), 
        "uso_cpu": cbrkit.sim.numbers.linear(max=100)
    },
    types={
        "error": "categorical",
        "memoria": "numeric",
        "uso_cpu": "numeric"
    }
)3. Configuración de Similitud (El "Cerebro")
Aquí es donde CBRKit brilla. Vamos a definir cómo comparar el texto y los números.

In [11]:
import cbrkit.sim.generic
import cbrkit.sim.numbers

print("Funciones en 'generic':", [f for f in dir(cbrkit.sim.generic) if not f.startswith('_')])
print("Funciones en 'numbers':", [f for f in dir(cbrkit.sim.numbers) if not f.startswith('_')])

Funciones en 'generic': ['Any', 'Sequence', 'SimPairFunc', 'ValueType', 'defaultdict', 'equality', 'table']
Funciones en 'numbers': ['Number', 'SimPairFunc', 'exponential', 'linear', 'linear_interval', 'math', 'sigmoid', 'threshold']


In [15]:
import cbrkit

# 1. Definimos las funciones de similitud locales que ya sabemos que funcionan
sim_error = cbrkit.sim.strings.levenshtein()
sim_memoria = cbrkit.sim.numbers.linear(max=12)
sim_cpu = cbrkit.sim.numbers.linear(max=100)

# 2. Creamos nuestra propia función de similitud global (Lógica de Negocio)
def similitud_global(caso, consulta):
    # Calculamos similitudes individuales
    s1 = sim_error(caso["error"], consulta["error"])
    s2 = sim_memoria(caso["memoria"], consulta["memoria"])
    s3 = sim_cpu(caso["uso_cpu"], consulta["uso_cpu"])
    
    # Aplicamos pesos (60% error, 20% memoria, 20% CPU)
    return (s1 * 0.6) + (s2 * 0.2) + (s3 * 0.2)

# 3. Construimos el retriever de CBRKit usando nuestra función
retriever = cbrkit.retrieval.build(similitud_global)

# 4. Datos y Ejecución
casos = {
    "C1": {"error": "pantalla_azul", "memoria": 8, "uso_cpu": 90},
    "C2": {"error": "lags", "memoria": 16, "uso_cpu": 40},
    "C3": {"error": "pantalla_azul", "memoria": 12, "uso_cpu": 85}
}
query = {"error": "pantalla_azul", "memoria": 10, "uso_cpu": 80}

result = cbrkit.retrieval.apply(casos, query, retriever)

print("¡Ranking calculado con éxito!")
print(result.ranking)

¡Ranking calculado con éxito!
['C3', 'C1', 'C2']


4. Ejecución y Resultados
CBRKit calculará la similitud global combinando todas las locales.

In [17]:
import cbrkit
import pandas as pd

# 1. Definimos métricas base
sim_error = cbrkit.sim.strings.levenshtein()
sim_memoria = cbrkit.sim.numbers.linear(max=16)
sim_cpu = cbrkit.sim.numbers.linear(max=100)

# 2. Función de similitud global (Sencilla y efectiva)
def similitud_global(caso, consulta):
    s1 = sim_error(caso["error"], consulta["error"])
    s2 = sim_memoria(caso["memoria"], consulta["memoria"])
    s3 = sim_cpu(caso["uso_cpu"], consulta["uso_cpu"])
    # Pesos: 60% error, 20% memoria, 20% cpu
    return (s1 * 0.6) + (s2 * 0.2) + (s3 * 0.2)

# 3. Datos
casos_data = {
    "C1": {"error": "pantalla_azul", "memoria": 8, "uso_cpu": 90, "solucion": "Reiniciar drivers"},
    "C2": {"error": "lags", "memoria": 4, "uso_cpu": 50, "solucion": "Limpiar temporales"},
    "C3": {"error": "pantalla_azul", "memoria": 12, "uso_cpu": 85, "solucion": "Cambiar RAM"}
}
query = {"error": "pantalla_azul", "memoria": 10, "uso_cpu": 80}

# 4. Recuperación (Usando 'apply' y 'build' que sí están en tu versión)
retriever = cbrkit.retrieval.build(similitud_global)
result = cbrkit.retrieval.apply(casos_data, query, retriever)

# 5. INTEGRACIÓN CON PANDAS
# Creamos un DataFrame con los datos originales
df_casos = pd.DataFrame.from_dict(casos_data, orient='index')

# Añadimos la columna de similitud obtenida del ranking
# result.ranking es un diccionario {id_caso: score}
df_casos['similitud'] = pd.Series(result.ranking)

# Ordenamos por similitud y mostramos
df_final = df_casos.sort_values(by='similitud', ascending=False)

print("Visualización del Prototipo CBR:")
df_final

Visualización del Prototipo CBR:


,error,memoria,uso_cpu,solucion,similitud
C1,pantalla_azul,8,90,Reiniciar drivers,NaN
C2,lags,4,50,Limpiar temporales,NaN
C3,pantalla_azul,12,85,Cambiar RAM,NaN


### El ciclo CBR completado
Con esto has implementado la fase de Retrieve (Recuperar). En un sistema real, ahora tomarías la "solución" del caso con mayor similitud (en este caso el que tenga el puntaje más cercano a 1.0) y se la presentarías al usuario.